In [13]:
# Cell 1: Imports, double precision, and device
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pickle, time

torch.set_default_dtype(torch.float64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [14]:
# Cell 2: Big, deep NN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, 100), nn.Tanh(),
            nn.Linear(100, 100), nn.Tanh(),
            nn.Linear(100, 100), nn.Tanh(),
            nn.Linear(100, 100), nn.Tanh(),
            nn.Linear(100, 1)
        )
    def forward(self, x):
        return self.layers(x)
model = Net().to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)
model.apply(init_weights)


Net(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Tanh()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Tanh()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): Tanh()
    (8): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [15]:
def exact_solution(x):
    # x: [N,2]
    # u(x1, x2) = x1^2 * x2^2 * (1 - x1)^2 * (1 - x2)^2
    return x[:,0]**2 * x[:,1]**2 * (1 - x[:,0])**2 * (1 - x[:,1])**2

def rhs_f(x):
    x.requires_grad_()
    u = exact_solution(x)

    # 1. First Laplacian (Delta u)
    grad_u = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    lap_u = sum([
        torch.autograd.grad(grad_u[:, i], x, torch.ones_like(grad_u[:, i]), create_graph=True)[0][:, i]
        for i in range(2)
    ])

    # 2. Second Laplacian (Delta^2 u) - Apply same logic to lap_u
    grad_lap = torch.autograd.grad(lap_u, x, torch.ones_like(lap_u), create_graph=True)[0]
    biharmonic = sum([
        torch.autograd.grad(grad_lap[:, i], x, torch.ones_like(grad_lap[:, i]), create_graph=True)[0][:, i]
        for i in range(2)
    ])

    return biharmonic.detach() # Detach because f is a constant constant ground truth
def dirichlet_bc(x):  # Dirichlet on all boundaries
    return exact_solution(x)


In [16]:
# Cell 4: Boundary sampler with normals
N_int, N_bd = 10000, 4000

def get_interior(N):
    return torch.rand(N, 2, device=device)

def get_boundary(M):
    grid = torch.linspace(0, 1, M//4, device=device)
    zeros = torch.zeros_like(grid)
    ones = torch.ones_like(grid)

    # Points on the 4 sides
    pts = [
        torch.stack([grid, zeros], dim=1), # Bottom (y=0)
        torch.stack([grid, ones], dim=1),  # Top (y=1)
        torch.stack([zeros, grid], dim=1), # Left (x=0)
        torch.stack([ones, grid], dim=1)   # Right (x=1)
    ]

    # Corresponding Normal vectors
    normals = [
        torch.stack([zeros, -ones], dim=1), # Bottom normal (0, -1)
        torch.stack([zeros, ones], dim=1),  # Top normal (0, 1)
        torch.stack([-ones, zeros], dim=1), # Left normal (-1, 0)
        torch.stack([ones, zeros], dim=1)   # Right normal (1, 0)
    ]

    return torch.cat(pts, dim=0), torch.cat(normals, dim=0)

In [17]:
# Cell 5: DRM Loss with Neumann Condition
def drm_loss(model, x_int, x_bd, n_bd, bc_weight=100.0):
    # --- 1. Domain Energy (Interior) ---
    x_int.requires_grad_()
    u = model(x_int)

    # Compute Laplacian via Autograd
    grad_u = torch.autograd.grad(u, x_int, torch.ones_like(u), create_graph=True)[0]
    lap_u = sum([
        torch.autograd.grad(grad_u[:, i], x_int, torch.ones_like(grad_u[:, i]), create_graph=True)[0][:, i]
        for i in range(2)
    ])

    # Compute Biharmonic (Laplacian of Laplacian)
    # grad_lap = torch.autograd.grad(lap_u, x_int, grad_outputs=torch.ones_like(lap_u), create_graph=True)[0]
    # biharmonic = sum([
    #    torch.autograd.grad(grad_lap[:, i], x_int, torch.ones_like(grad_lap[:, i]), create_graph=True)[0][:, i]
    #    for i in range(2)
    # ])

    f = rhs_f(x_int)
    energy = ((0.5 * lap_u ** 2 - f * u.squeeze()).mean())

    # --- 2. Boundary Conditions ---
    # We need gradients on boundary points to compute normal derivative
    x_bd.requires_grad_()
    u_bd = model(x_bd).squeeze()

    # A. Dirichlet Loss (u = 0 on boundary)
    # Note: dirichlet_bc(x_bd) returns 0 for this exact solution
    bc_dir_loss = ((u_bd - dirichlet_bc(x_bd))**2).mean()

   # B. Neumann Target (g2 = du_exact / dn)
    # -- Calculate Exact Normal Derivative --
    u_exact_bd = exact_solution(x_bd)
    grad_exact_bd = torch.autograd.grad(u_exact_bd, x_bd, torch.ones_like(u_exact_bd), create_graph=True)[0]
    target_neumann = (grad_exact_bd * n_bd).sum(dim=1).detach() # This will be 0.0 for this problem

    # -- Calculate Predicted Normal Derivative --
    grad_u_bd = torch.autograd.grad(u_bd, x_bd, torch.ones_like(u_bd), create_graph=True)[0]
    du_dn_pred = (grad_u_bd * n_bd).sum(dim=1)

    # -- Neumann Loss --
    bc_neu_loss = ((du_dn_pred - target_neumann)**2).mean()

    return energy + bc_weight * (bc_dir_loss + bc_neu_loss)

In [ ]:
# Cell 6: Training
epochs = 12000
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5000, gamma=0.3)
losses = []
start_time = time.time()

for epoch in range(epochs):
    # Unpack normals here
    x_int, (x_bd, n_bd) = get_interior(N_int), get_boundary(N_bd)

    # Pass normals to loss
    loss = drm_loss(model, x_int, x_bd, n_bd, bc_weight=100.0)

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.5)
    optimizer.step()
    scheduler.step()
    losses.append(loss.item())

    if epoch % 500 == 0:
        print(f"Epoch {epoch}: Loss {loss.item():.6e}")

# Final LBFGS
lbfgs = torch.optim.LBFGS(model.parameters(), lr=0.5, max_iter=250)
def closure():
    x_int, (x_bd, n_bd) = get_interior(N_int), get_boundary(N_bd)
    loss = drm_loss(model, x_int, x_bd, n_bd, bc_weight=4000.0)
    lbfgs.zero_grad()
    loss.backward()
    return loss
lbfgs.step(closure)
train_time = time.time() - start_time

Epoch 0: Loss 2.966447e+00
Epoch 500: Loss -2.821252e-03
Epoch 1000: Loss -3.371073e-03
Epoch 1500: Loss -6.827790e-03
Epoch 2000: Loss -6.547862e-03
Epoch 2500: Loss -6.590160e-03
Epoch 3000: Loss -6.677724e-03
Epoch 3500: Loss -6.581500e-03
Epoch 4000: Loss -6.648561e-03
Epoch 4500: Loss -6.772307e-03


In [ ]:
# Cell 7: Save outputa
torch.save(model.state_dict(), "drm_model_q2.pt")

In [ ]:
# Cell 8: Print arch/settings
print(f"Training time: {train_time:.2f} s")
print("NN Architecture:\n", model)
n_L = sum(torch.count_nonzero(p).item() for p in model.parameters())
print("Total nonzero parameters n_L:", n_L)
print("Adam lr: 5e-4, LBFGS steps: 250")
print("Interior pts:", N_int, "Boundary pts:", N_bd, "Boundary penalty:", 100.0)
print(f"Final training loss: {losses[-1]:.6e}")


In [ ]:
# Cell 9: Plot solution/error
gsize = 61
xg = np.linspace(0, 1, gsize)
mg = np.meshgrid(xg, xg)
gp = torch.tensor(np.stack([mg[0].ravel(), mg[1].ravel()], axis=-1), device=device, dtype=torch.float64)

with torch.no_grad():
    u_nn = model(gp).detach().cpu().numpy().reshape(gsize, gsize)
    u_gt = exact_solution(gp).detach().cpu().numpy().reshape(gsize, gsize)
    err = np.abs(u_nn - u_gt)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(mg[0], mg[1], u_nn, cmap='jet')
plt.title('NN Solution')
plt.savefig("NNsolution2.png")
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(mg[0], mg[1], u_gt, cmap='jet')
plt.title('GT Solution')
plt.savefig("GTsol2.png")
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(mg[0], mg[1], err, cmap='jet')
plt.title('|u-u_theta| Error')
plt.savefig("Error2.png")
plt.show()


In [ ]:
## Cell 10: Error calculation (careful detaching!)
gp.requires_grad_()
u_gt1d = exact_solution(gp).detach().cpu().numpy()
u_nn1d = model(gp).detach().cpu().numpy()

u_gt_val = exact_solution(gp)
u_nn_val = model(gp)
u_gt_grad = torch.autograd.grad(u_gt_val, gp, torch.ones_like(u_gt_val), create_graph=True)[0]
u_nn_grad = torch.autograd.grad(u_nn_val, gp, torch.ones_like(u_nn_val), create_graph=True)[0]
u_gt_grad_np = u_gt_grad.detach().cpu().numpy()
u_nn_grad_np = u_nn_grad.detach().cpu().numpy()
H_gt = [torch.autograd.grad(u_gt_grad[:, i], gp, torch.ones_like(u_gt_grad[:, i]), create_graph=True)[0][:, i].detach().cpu().numpy() for i in range(2)]
H_nn = [torch.autograd.grad(u_nn_grad[:, i], gp, torch.ones_like(u_nn_grad[:, i]), create_graph=True)[0][:, i].detach().cpu().numpy() for i in range(2)]
def H2norm(H): return np.sqrt(sum([np.mean(h**2) for h in H]))
def H1norm(grad): return np.sqrt(np.mean(grad[:,0]**2 + grad[:,1]**2))

L2Err = np.sqrt(np.mean((u_gt1d-u_nn1d)**2))
RelL2Err = L2Err / np.sqrt(np.mean(u_gt1d**2))
energyErr = L2Err + H1norm(u_gt_grad_np-u_nn_grad_np)
relEnergyErr = energyErr / (np.sqrt(np.mean(u_gt1d**2)) + H1norm(u_gt_grad_np))
H2Err = H2norm([H_gt[0]-H_nn[0], H_gt[1]-H_nn[1]])
relH2Err = H2Err / H2norm(H_gt)

print(f"L2 Error: {L2Err:.3e}")
print(f"Relative L2 Error: {RelL2Err:.3e}")
print(f"Energy Error (L2 + H1): {energyErr:.3e}")
print(f"Relative Energy Error (H1Relative): {relEnergyErr:.3e}")
print(f"H2 Error: {H2Err:.3e}")
print(f"Relative H2 Error: {relH2Err:.3e}")


In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.yscale('log')  # <- THIS is the key change!
plt.title("Training Loss History (Log Scale)")
plt.savefig("LossHistory_log_2.png")
plt.show()